# This code is designed to explore the hemodynamics of the LSNM

In [1]:
import numpy as np
from scipy.integrate import odeint

In [2]:
# define physiological balloon model parameters...
tau_s = 1.54	# rate constant of vasodilatory signal decay in seconds
		# from Heinzle 2016, page 559 (table 1)

tau_f = 2.44	# Time of flow-dependent elimination or feedback regulation
		# in seconds, from Heinzle 2016, page 559 (text)

alpha = 0.32	# Grubb's vessel stiffness exponent, Heinzle 2016, page 559 (text)
		# also Havlicek 2015, page 357 (table 1A)		     

tau_0 = 2.0	# Hemodynamic transit time in seconds
		# Havlicek 2015, page 357 (table 1A)
		# Also Obata 2004, page 146 (text)

epsilon = 0.1   # efficacy of synaptic activity to induce the signal,
                # from Friston et al, 2000

E_0 = 0.34	# Resting oxygen extraction fraction from Heinzle 2016, page 559 (text)
		# also mentioned on page 569 with references to Obata 2004 and Stephan 2007
		# reasonable values are between 0.3 and 0.4 

In [3]:
##############################################################################                 
#
# define gradient echo BOLD model parameters... at 7T
#

r_0 = 340.0	# Slope of intravascular relaxation rate (Hz)
		# this doesn't matter at 7T because e is zero
		# a value of 325 Hz is mentions by Heinzle 2016, page 568 (text)
		# referece to Ivanov 2013 of in vivo measurement of 360 Hz in humans
		# Heinzle recommends a value of 340 Hz

nu_0 = 188.1	# Frequency offset at the outer surface of magnetized vessels (Hz) 
		# Havlicek 2015, page 360 (table 1b)

e = 0.026	# Ratio of intra- and extravascular BOLD signal at is approximately zero at 7T
		# Heinzle 2016, page 569 (text) suggest a value of 0.026 at 7T
		# Uludag et al. 2009, also see Havlicek 2015, page 360 (table 1B)

V_0 = 0.02	# Resting blood volume fraction, from Obata 2004, page 146 (text)
		# this is merely a scaling factor of the BOLD signal, so ultimately we 
		# aren't very worried by this refer to Havlicek 2015, page 359 (text)

TE = 0.025	# Suggested value by Heinzle 2016, page 569 
		# .022 Echo time for a 7T scanner Havlicek 2015, Supplementary Material S4
		# also reference to Uludag 2009, page 152 (text) though this seems to be 28 ms at 7T and 22 ms at 9.4T

In [4]:
# here I have modified the coupling to the parameters suggested from Heinzle on page 
lambda_d_s = 0.0 # supragranular draining coupling parameter set to 0.0 because this is the top layer
lambda_4_d = 0.5 # layer 4 draining coupling parameter set to 1.0 in Heinzle paper
lambda_d_d = 0.5 # infragranular draining coupling parameter set to 1.0 in Heinzle paper
tau_d = 0.5 # draining time constant this is halved from what is suggested by Heinzle because we have three layers, however it is possible that tau_d would vary between layers

In [5]:
# calculate BOLD model coefficients, from Friston et al (2000)
k1 = 4.3 * nu_0 * E_0 * TE
k2 = e * r_0 * E_0 * TE
k3 = 1.0 - e

In [6]:
def Heinzle_function(y, t_DE, syn):
    ''' 
    Heinzle model of hemodynamic change
    Heinzle, J., Koopmans, P.J., den Ouden, H.E., Raman, S. and Stephan, K.E., 2016.
    A hemodynamic model for layered BOLD signals. Neuroimage, 125, pp.556-570.
    '''
    
    # unpack initial values
    s_s = y[0]
    f_s = y[1]
    v_s = y[2]
    q_s = y[3]
    v_star_s = y[4]
    q_star_s = y[5]
    s_4 = y[6]
    f_4 = y[7]
    v_4 = y[8]
    q_4 = y[9]
    v_star_4 = y[10]
    q_star_4 = y[11]
    s_d = y[12]
    f_d = y[13]
    v_d = y[14]
    q_d = y[15]
    v_star_d = y[16]
    q_star_d = y[17]
    # unpack laminar synaptic activity into supragranular, layer 4, and infragranular
    x_s = syn[0,:]
    x_4 = syn[1,:]
    x_d = syn[2,:]

    if (t_DE < T):
    	xx_s = x_s[int(np.floor(t_DE * synaptic_timesteps / T))]
        #print xx_s
    	xx_4 = x_4[int(np.floor(t_DE * synaptic_timesteps / T))]
        #print xx_4
    	xx_d = x_d[int(np.floor(t_DE * synaptic_timesteps / T))]
        #print xx_d
    else:
        xx_s = x_s[synaptic_timesteps-1]
        #print xx_s
        xx_4 = x_4[synaptic_timesteps-1]
        #print xx_4
        xx_d = x_d[synaptic_timesteps-1]
        #print xx_d

    # the balloon model equations for supragranular layer
    ds_s = epsilon * xx_s - (1. / tau_s) * s_s - (1. / tau_f) * (f_s - 1)
    df_s = s_s
    dv_s = (1. / tau_0) * (f_s - v_s ** (1. / alpha) + lambda_4_d * v_star_4)
    dq_s = (1. / tau_0) * ((f_s * (1. - (1. - E_0) ** (1. / f_s)) / E_0) -
                          (v_s ** (1. / alpha)) * (q_s / v_s) + lambda_4_d * q_star_4)
    dv_star_s = (1/tau_d) * (v_s - v_star_s - 1)
    dq_star_s = (1/tau_d) * (q_s - q_star_s - 1)
    # the balloon model equations for layer 4
    ds_4 = epsilon * xx_4 - (1. / tau_s) * s_4 - (1. / tau_f) * (f_4 - 1)
    df_4 = s_4
    dv_4 = (1. / tau_0) * (f_4 - v_4 ** (1. / alpha) + lambda_d_d * v_star_d)
    dq_4 = (1. / tau_0) * ((f_4 * (1. - (1. - E_0) ** (1. / f_4)) / E_0) -
                          (v_4 ** (1. / alpha)) * (q_4 / v_4) + lambda_d_d * q_star_d)
    dv_star_4 = (1/tau_d) * (v_4 - v_star_4 - 1)
    dq_star_4 = (1/tau_d) * (q_4 - q_star_4 - 1)
    # the balloon model equations for infragranular layer
    ds_d = epsilon * xx_d - (1. / tau_s) * s_d - (1. / tau_f) * (f_d - 1)
    df_d = s_d
    dv_d = (1. / tau_0) * (f_d - v_d ** (1. / alpha))
    dq_d = (1. / tau_0) * ((f_d * (1. - (1. - E_0) ** (1. / f_d)) / E_0) -
                          (v_d ** (1. / alpha)) * (q_d / v_d))
    dv_star_d = (1. / tau_d) * (v_d - v_star_d - 1)
    dq_star_d = (1. / tau_d) * (q_d - q_star_d - 1)


    return [ds_s, df_s, dv_s, dq_s, dv_star_s, dq_star_s, ds_4, df_4, dv_4, dq_4, dv_star_4, dq_star_4, ds_d, df_d, dv_d, dq_d, dv_star_d, dq_star_d]

In [7]:
# define neural synaptic time interval in seconds. The simulation data is collected
# one data point at synaptic intervals (10 simulation timesteps). Every simulation
# timestep is equivalent to 5 ms.
Ti = 0.005 * 10

# Total time of scanning experiment in seconds 
#T = 198
T = 200

# the scanning happened every Tr interval below (in milliseconds). This
# is the time needed to sample hemodynamic activity to produce
# each fMRI image.
Tr = 2

# how many scans do you want to remove from beginning of BOLD timeseries?
scans_to_remove = 5
# this is the start time so the normalization isn't as messed up, ie low numbers at beginning
start_time = 5*Tr/Ti

In [8]:
syn = np.load('WK_DMS_ISA.npy')

In [9]:
ts_length = syn[0].size
# Throw away first value of each synaptic array (it is always zero)
v1_syn_s = syn[0]
v1_syn_4 = syn[1]
v1_syn_d = syn[2]
v4_syn_s = syn[3]
v4_syn_4 = syn[4]
v4_syn_d = syn[5]
it_syn_s = syn[6]
it_syn_4 = syn[7]
it_syn_d = syn[8]
fs_syn_s = syn[9]
fs_syn_4 = syn[10]
fs_syn_d = syn[11]
d1_syn_s = syn[12]
d1_syn_4 = syn[13]
d1_syn_d = syn[14]
d2_syn_s = syn[15]
d2_syn_4 = syn[16]
d2_syn_d = syn[17]
fr_syn_s = syn[18]
fr_syn_4 = syn[19]
fr_syn_d = syn[20]
v1_syn_s_min = v1_syn_s[int(start_time):ts_length].min()
v1_syn_s_max = v1_syn_s[int(start_time):ts_length].max()
v1_syn_4_min = v1_syn_4[int(start_time):ts_length].min()
v1_syn_4_max = v1_syn_4[int(start_time):ts_length].max()
v1_syn_d_min = v1_syn_d[int(start_time):ts_length].min()
v1_syn_d_max = v1_syn_d[int(start_time):ts_length].max()

v4_syn_s_min = v4_syn_s[int(start_time):ts_length].min()
v4_syn_s_max = v4_syn_s[int(start_time):ts_length].max()
v4_syn_4_min = v4_syn_4[int(start_time):ts_length].min()
v4_syn_4_max = v4_syn_4[int(start_time):ts_length].max()
v4_syn_d_min = v4_syn_d[int(start_time):ts_length].min()
v4_syn_d_max = v4_syn_d[int(start_time):ts_length].max()

it_syn_s_min = it_syn_s[int(start_time):ts_length].min()
it_syn_s_max = it_syn_s[int(start_time):ts_length].max()
it_syn_4_min = it_syn_4[int(start_time):ts_length].min()
it_syn_4_max = it_syn_4[int(start_time):ts_length].max()
it_syn_d_min = it_syn_d[int(start_time):ts_length].min()
it_syn_d_max = it_syn_d[int(start_time):ts_length].max()

fs_syn_s_min = fs_syn_s[int(start_time):ts_length].min()
fs_syn_s_max = fs_syn_s[int(start_time):ts_length].max()
fs_syn_4_min = fs_syn_4[int(start_time):ts_length].min()
fs_syn_4_max = fs_syn_4[int(start_time):ts_length].max()
fs_syn_d_min = fs_syn_d[int(start_time):ts_length].min()
fs_syn_d_max = fs_syn_d[int(start_time):ts_length].max()

d1_syn_s_min = d1_syn_s[int(start_time):ts_length].min()
d1_syn_s_max = d1_syn_s[int(start_time):ts_length].max()
d1_syn_4_min = d1_syn_4[int(start_time):ts_length].min()
d1_syn_4_max = d1_syn_4[int(start_time):ts_length].max()
d1_syn_d_min = d1_syn_d[int(start_time):ts_length].min()
d1_syn_d_max = d1_syn_d[int(start_time):ts_length].max()

d2_syn_s_min = d2_syn_s[int(start_time):ts_length].min()
d2_syn_s_max = d2_syn_s[int(start_time):ts_length].max()
d2_syn_4_min = d2_syn_4[int(start_time):ts_length].min()
d2_syn_4_max = d2_syn_4[int(start_time):ts_length].max()
d2_syn_d_min = d2_syn_d[int(start_time):ts_length].min()
d2_syn_d_max = d2_syn_d[int(start_time):ts_length].max()

fr_syn_s_min = fr_syn_s[int(start_time):ts_length].min()
fr_syn_s_max = fr_syn_s[int(start_time):ts_length].max()
fr_syn_4_min = fr_syn_4[int(start_time):ts_length].min()
fr_syn_4_max = fr_syn_4[int(start_time):ts_length].max()
fr_syn_d_min = fr_syn_d[int(start_time):ts_length].min()
fr_syn_d_max = fr_syn_d[int(start_time):ts_length].max()

# min-max scaling: maps data into range [0,1]
v1_syn_s = (v1_syn_s-v1_syn_s_min) / (v1_syn_s_max - v1_syn_s_min)
v1_syn_4 = (v1_syn_4-v1_syn_4_min) / (v1_syn_4_max - v1_syn_4_min)
v1_syn_d = (v1_syn_d-v1_syn_d_min) / (v1_syn_d_max - v1_syn_d_min)
v4_syn_s = (v4_syn_s-v4_syn_s_min) / (v4_syn_s_max - v4_syn_s_min)
v4_syn_4 = (v4_syn_4-v4_syn_4_min) / (v4_syn_4_max - v4_syn_4_min)
v4_syn_d = (v4_syn_d-v4_syn_d_min) / (v4_syn_d_max - v4_syn_d_min)
it_syn_s = (it_syn_s-it_syn_s_min) / (it_syn_s_max - it_syn_s_min)
it_syn_4 = (it_syn_4-it_syn_4_min) / (it_syn_4_max - it_syn_4_min)
it_syn_d = (it_syn_d-it_syn_d_min) / (it_syn_d_max - it_syn_d_min)
fs_syn_s = (fs_syn_s-fs_syn_s_min) / (fs_syn_s_max - fs_syn_s_min)
fs_syn_4 = (fs_syn_4-fs_syn_4_min) / (fs_syn_4_max - fs_syn_4_min)
fs_syn_d = (fs_syn_d-fs_syn_d_min) / (fs_syn_d_max - fs_syn_d_min)
d1_syn_s = (d1_syn_s-d1_syn_s_min) / (d1_syn_s_max - d1_syn_s_min)
d1_syn_4 = (d1_syn_4-d1_syn_4_min) / (d1_syn_4_max - d1_syn_4_min)
d1_syn_d = (d1_syn_d-d1_syn_d_min) / (d1_syn_d_max - d1_syn_d_min)
d2_syn_s = (d2_syn_s-d2_syn_s_min) / (d2_syn_s_max - d2_syn_s_min)
d2_syn_4 = (d2_syn_4-d2_syn_4_min) / (d2_syn_4_max - d2_syn_4_min)
d2_syn_d = (d2_syn_d-d2_syn_d_min) / (d2_syn_d_max - d2_syn_d_min)
fr_syn_s = (fr_syn_s-fr_syn_s_min) / (fr_syn_s_max - fr_syn_s_min)
fr_syn_4 = (fr_syn_4-fr_syn_4_min) / (fr_syn_4_max - fr_syn_4_min)
fr_syn_d = (fr_syn_d-fr_syn_d_min) / (fr_syn_d_max - fr_syn_d_min)

# load up the regional data
v1_syn = np.array([v1_syn_s, v1_syn_4, v1_syn_d])
v4_syn = np.array([v4_syn_s, v4_syn_4, v4_syn_d])
it_syn = np.array([it_syn_s, it_syn_4, it_syn_d])
fs_syn = np.array([fs_syn_s, fs_syn_4, fs_syn_d])
d1_syn = np.array([d1_syn_s, d1_syn_4, d1_syn_d])
d2_syn = np.array([d2_syn_s, d2_syn_4, d2_syn_d])
fr_syn = np.array([fr_syn_s, fr_syn_4, fr_syn_d])

synaptic_timesteps = v1_syn_s.size
time_in_seconds = np.arange(0, T, Ti)

In [10]:
# Hard coded initial conditions
s = 0   # s, blood flow
f = 1.  # f, blood inflow
v = 1.  # v, venous blood volume
q = 1.  # q, deoxyhemoglobin content
v_star = 1. # v_star, delayed venous blood volume
q_star = 1. # q_star, diluted deoxyhemoglobin content

In [11]:
# initial conditions vectors
y_0_v1 = np.array([s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star])
y_0_v4 = np.array([s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star])
y_0_it = np.array([s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star])
y_0_fs = np.array([s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star])
y_0_d1 = np.array([s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star])
y_0_d2 = np.array([s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star])
y_0_fr = np.array([s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star, s, f, v, q, v_star, q_star])

In [12]:
# generate synaptic time array
t_syn = np.arange(0, synaptic_timesteps)
# generate time array for solution
t = time_in_seconds

In [13]:
# solve the ODE's
state_v1 = odeint(Heinzle_function, y_0_v1, t, args=(v1_syn,) )
state_v4 = odeint(Heinzle_function, y_0_v4, t, args=(v4_syn,) )
state_it = odeint(Heinzle_function, y_0_it, t, args=(it_syn,) )
state_fs = odeint(Heinzle_function, y_0_fs, t, args=(fs_syn,) )
state_d1 = odeint(Heinzle_function, y_0_d1, t, args=(d1_syn,) )
state_d2 = odeint(Heinzle_function, y_0_d2, t, args=(d2_syn,) )
state_fr = odeint(Heinzle_function, y_0_fr, t, args=(fr_syn,) )

In [14]:
# Unpack the state variables used in the BOLD model
s_v1_s = state_v1[:, 0]
f_v1_s = state_v1[:, 1]
v_v1_s = state_v1[:, 2]
q_v1_s = state_v1[:, 3]
v_star_v1_s = state_v1[:, 4]
q_star_v1_s = state_v1[:, 5]       
s_v1_4 = state_v1[:, 6]
f_v1_4 = state_v1[:, 7]
v_v1_4 = state_v1[:, 8]
q_v1_4 = state_v1[:, 9]
v_star_v1_4 = state_v1[:, 10]
q_star_v1_4 = state_v1[:, 11]          
s_v1_d = state_v1[:, 12]
f_v1_d = state_v1[:, 13]
v_v1_d = state_v1[:, 14]
q_v1_d = state_v1[:, 15]   
v_star_v1_4 = state_v1[:, 16]
q_star_v1_4 = state_v1[:, 17]

s_v4_s = state_v4[:, 0]
f_v4_s = state_v4[:, 1]
v_v4_s = state_v4[:, 2]
q_v4_s = state_v4[:, 3]
v_star_v4_s = state_v4[:, 4]
q_star_v4_s = state_v4[:, 5]       
s_v4_4 = state_v4[:, 6]
f_v4_4 = state_v4[:, 7]
v_v4_4 = state_v4[:, 8]
q_v4_4 = state_v4[:, 9]
v_star_v4_4 = state_v4[:, 10]
q_star_v4_4 = state_v4[:, 11]          
s_v4_d = state_v4[:, 12]
f_v4_d = state_v4[:, 13]
v_v4_d = state_v4[:, 14]
q_v4_d = state_v4[:, 15]   
v_star_v4_4 = state_v4[:, 16]
q_star_v4_4 = state_v4[:, 17]
       
s_it_s = state_it[:, 0]
f_it_s = state_it[:, 1]
v_it_s = state_it[:, 2]
q_it_s = state_it[:, 3]
v_star_it_s = state_it[:, 4]
q_star_it_s = state_it[:, 5]       
s_it_4 = state_it[:, 6]
f_it_4 = state_it[:, 7]
v_it_4 = state_it[:, 8]
q_it_4 = state_it[:, 9]
v_star_it_4 = state_it[:, 10]
q_star_it_4 = state_it[:, 11]          
s_it_d = state_it[:, 12]
f_it_d = state_it[:, 13]
v_it_d = state_it[:, 14]
q_it_d = state_it[:, 15]   
v_star_it_4 = state_it[:, 16]
q_star_it_4 = state_it[:, 17]

s_fs_s = state_fs[:, 0]
f_fs_s = state_fs[:, 1]
v_fs_s = state_fs[:, 2]
q_fs_s = state_fs[:, 3]
v_star_fs_s = state_fs[:, 4]
q_star_fs_s = state_fs[:, 5]       
s_fs_4 = state_fs[:, 6]
f_fs_4 = state_fs[:, 7]
v_fs_4 = state_fs[:, 8]
q_fs_4 = state_fs[:, 9]
v_star_fs_4 = state_fs[:, 10]
q_star_fs_4 = state_fs[:, 11]          
s_fs_d = state_fs[:, 12]
f_fs_d = state_fs[:, 13]
v_fs_d = state_fs[:, 14]
q_fs_d = state_fs[:, 15]   
v_star_fs_4 = state_fs[:, 16]
q_star_fs_4 = state_fs[:, 17]

s_d1_s = state_d1[:, 0]
f_d1_s = state_d1[:, 1]
v_d1_s = state_d1[:, 2]
q_d1_s = state_d1[:, 3]
v_star_d1_s = state_d1[:, 4]
q_star_d1_s = state_d1[:, 5]       
s_d1_4 = state_d1[:, 6]
f_d1_4 = state_d1[:, 7]
v_d1_4 = state_d1[:, 8]
q_d1_4 = state_d1[:, 9]
v_star_d1_4 = state_d1[:, 10]
q_star_d1_4 = state_d1[:, 11]          
s_d1_d = state_d1[:, 12]
f_d1_d = state_d1[:, 13]
v_d1_d = state_d1[:, 14]
q_d1_d = state_d1[:, 15]   
v_star_d1_4 = state_d1[:, 16]
q_star_d1_4 = state_d1[:, 17]

s_d2_s = state_d2[:, 0]
f_d2_s = state_d2[:, 1]
v_d2_s = state_d2[:, 2]
q_d2_s = state_d2[:, 3]
v_star_d2_s = state_d2[:, 4]
q_star_d2_s = state_d2[:, 5]       
s_d2_4 = state_d2[:, 6]
f_d2_4 = state_d2[:, 7]
v_d2_4 = state_d2[:, 8]
q_d2_4 = state_d2[:, 9]
v_star_d2_4 = state_d2[:, 10]
q_star_d2_4 = state_d2[:, 11]          
s_d2_d = state_d2[:, 12]
f_d2_d = state_d2[:, 13]
v_d2_d = state_d2[:, 14]
q_d2_d = state_d2[:, 15]   
v_star_d2_4 = state_d2[:, 16]
q_star_d2_4 = state_d2[:, 17]

s_fr_s = state_fr[:, 0]
f_fr_s = state_fr[:, 1]
v_fr_s = state_fr[:, 2]
q_fr_s = state_fr[:, 3]
v_star_fr_s = state_fr[:, 4]
q_star_fr_s = state_fr[:, 5]       
s_fr_4 = state_fr[:, 6]
f_fr_4 = state_fr[:, 7]
v_fr_4 = state_fr[:, 8]
q_fr_4 = state_fr[:, 9]
v_star_fr_4 = state_fr[:, 10]
q_star_fr_4 = state_fr[:, 11]          
s_fr_d = state_fr[:, 12]
f_fr_d = state_fr[:, 13]
v_fr_d = state_fr[:, 14]
q_fr_d = state_fr[:, 15]   
v_star_fr_4 = state_fr[:, 16]
q_star_fr_4 = state_fr[:, 17]

In [15]:
# now, we need to calculate BOLD signal at each timestep, based on v and q obtained from solving
# Heinzle model ODE above.
v1_BOLD_s = np.array(V_0 * (k1 * (1. - q_v1_s) + k2 * (1. - q_v1_s / v_v1_s) + k3 * (1. - v_v1_s)) )
v1_BOLD_4 = np.array(V_0 * (k1 * (1. - q_v1_4) + k2 * (1. - q_v1_4 / v_v1_4) + k3 * (1. - v_v1_4)) )
v1_BOLD_d = np.array(V_0 * (k1 * (1. - q_v1_d) + k2 * (1. - q_v1_d / v_v1_d) + k3 * (1. - v_v1_d)) )

v4_BOLD_s = np.array(V_0 * (k1 * (1. - q_v4_s) + k2 * (1. - q_v4_s / v_v4_s) + k3 * (1. - v_v4_s)) )
v4_BOLD_4 = np.array(V_0 * (k1 * (1. - q_v4_4) + k2 * (1. - q_v4_4 / v_v4_4) + k3 * (1. - v_v4_4)) )
v4_BOLD_d = np.array(V_0 * (k1 * (1. - q_v4_d) + k2 * (1. - q_v4_d / v_v4_d) + k3 * (1. - v_v4_d)) )

it_BOLD_s = np.array(V_0 * (k1 * (1. - q_it_s) + k2 * (1. - q_it_s / v_it_s) + k3 * (1. - v_it_s)) )
it_BOLD_4 = np.array(V_0 * (k1 * (1. - q_it_4) + k2 * (1. - q_it_4 / v_it_4) + k3 * (1. - v_it_4)) )
it_BOLD_d = np.array(V_0 * (k1 * (1. - q_it_d) + k2 * (1. - q_it_d / v_it_d) + k3 * (1. - v_it_d)) )

fs_BOLD_s = np.array(V_0 * (k1 * (1. - q_fs_s) + k2 * (1. - q_fs_s / v_fs_s) + k3 * (1. - v_fs_s)) )
fs_BOLD_4 = np.array(V_0 * (k1 * (1. - q_fs_4) + k2 * (1. - q_fs_4 / v_fs_4) + k3 * (1. - v_fs_4)) )
fs_BOLD_d = np.array(V_0 * (k1 * (1. - q_fs_d) + k2 * (1. - q_fs_d / v_fs_d) + k3 * (1. - v_fs_d)) )

d1_BOLD_s = np.array(V_0 * (k1 * (1. - q_d1_s) + k2 * (1. - q_d1_s / v_d1_s) + k3 * (1. - v_d1_s)) )
d1_BOLD_4 = np.array(V_0 * (k1 * (1. - q_d1_4) + k2 * (1. - q_d1_4 / v_d1_4) + k3 * (1. - v_d1_4)) )
d1_BOLD_d = np.array(V_0 * (k1 * (1. - q_d1_d) + k2 * (1. - q_d1_d / v_d1_d) + k3 * (1. - v_d1_d)) )

d2_BOLD_s = np.array(V_0 * (k1 * (1. - q_d2_s) + k2 * (1. - q_d2_s / v_d2_s) + k3 * (1. - v_d2_s)) )
d2_BOLD_4 = np.array(V_0 * (k1 * (1. - q_d2_4) + k2 * (1. - q_d2_4 / v_d2_4) + k3 * (1. - v_d2_4)) )
d2_BOLD_d = np.array(V_0 * (k1 * (1. - q_d2_d) + k2 * (1. - q_d2_d / v_d2_d) + k3 * (1. - v_d2_d)) )

fr_BOLD_s = np.array(V_0 * (k1 * (1. - q_fr_s) + k2 * (1. - q_fr_s / v_fr_s) + k3 * (1. - v_fr_s)) )
fr_BOLD_4 = np.array(V_0 * (k1 * (1. - q_fr_4) + k2 * (1. - q_fr_4 / v_fr_4) + k3 * (1. - v_fr_4)) )
fr_BOLD_d = np.array(V_0 * (k1 * (1. - q_fr_d) + k2 * (1. - q_fr_d / v_fr_d) + k3 * (1. - v_fr_d)) )

In [16]:
# put the hemodynamic values in numpy arrays
lsnm_lam_flow = np.array([t,f_v1_s, f_v1_4, f_v1_d, f_v4_s, f_v4_4, f_v4_d,
		f_it_s, f_it_4, f_v1_d, f_fs_s, f_fs_4, f_fs_d,
		f_d1_s, f_d1_4, f_d1_d, f_d2_s, f_d2_4, f_d2_d,
		f_fr_s, f_fr_4, f_fr_d])
lsnm_lam_deoxy = np.array([t,q_v1_s, q_v1_4, q_v1_d, q_v4_s, q_v4_4, q_v4_d,
		q_it_s, q_it_4, q_v1_d, q_fs_s, q_fs_4, q_fs_d,
		q_d1_s, q_d1_4, q_d1_d, q_d2_s, q_d2_4, q_d2_d,
		q_fr_s, q_fr_4, q_fr_d])
lsnm_lam_volume = np.array([t,v_v1_s, v_v1_4, v_v1_d, v_v4_s, v_v4_4, v_v4_d,
		v_it_s, v_it_4, v_v1_d, v_fs_s, v_fs_4, v_fs_d,
		v_d1_s, v_d1_4, v_d1_d, v_d2_s, v_d2_4, v_d2_d,
		v_fr_s, v_fr_4, v_fr_d])
lsnm_BOLD_not_downsampled = np.array([t,v1_BOLD_s, v1_BOLD_4, v1_BOLD_d,v4_BOLD_s, v4_BOLD_4, v4_BOLD_d,
                      it_BOLD_s, it_BOLD_4, it_BOLD_d,
                      fs_BOLD_s, fs_BOLD_4, fs_BOLD_d,
                      d1_BOLD_s, d1_BOLD_4, d1_BOLD_d, 
                      d2_BOLD_s, d2_BOLD_4, d2_BOLD_d,
                      fr_BOLD_s, fr_BOLD_4, fr_BOLD_d
                      ])

In [17]:
# Save the files in the local directory for later use in display purposes
flow_name = 'regional_laminar_blood_flow_check_7T_drain_DMS.npy'
deoxy_name = 'regional_laminar_deoxy_check_7T_drain_DMS.npy'
volume_name = 'regional_laminar_volume_check_7T_drain_DMS.npy'
lsnm_BOLD_not_downsampled_file_name = 'WK_laminar_LSNM_bold_balloon_7T_drain_DMS_not_downsampled.npy'
np.save(flow_name, lsnm_lam_flow)
np.save(deoxy_name, lsnm_lam_deoxy)
np.save(volume_name, lsnm_lam_volume)
np.save(lsnm_BOLD_not_downsampled_file_name,lsnm_BOLD_not_downsampled)

In [18]:
# downsample the BOLD signal arrays to produce scan rate of 2 per second (or whatever TR you decide to use)
scanning_timescale = np.arange(0, synaptic_timesteps, ts_length / (T/Tr))
scanning_timescale = scanning_timescale.astype(int)
# downsample the BOLD signal
mr_time = t[scanning_timescale]
v1_BOLD_s = v1_BOLD_s[scanning_timescale]
v1_BOLD_4 = v1_BOLD_4[scanning_timescale]
v1_BOLD_d = v1_BOLD_d[scanning_timescale]
v4_BOLD_s = v4_BOLD_s[scanning_timescale]
v4_BOLD_4 = v4_BOLD_4[scanning_timescale]
v4_BOLD_d = v4_BOLD_d[scanning_timescale]
it_BOLD_s = it_BOLD_s[scanning_timescale]
it_BOLD_4 = it_BOLD_4[scanning_timescale]
it_BOLD_d = it_BOLD_d[scanning_timescale]
d1_BOLD_s = d1_BOLD_s[scanning_timescale]
d1_BOLD_4 = d1_BOLD_4[scanning_timescale]
d1_BOLD_d = d1_BOLD_d[scanning_timescale]
d2_BOLD_s = d2_BOLD_s[scanning_timescale]
d2_BOLD_4 = d2_BOLD_4[scanning_timescale]
d2_BOLD_d = d2_BOLD_d[scanning_timescale]
fs_BOLD_s = fs_BOLD_s[scanning_timescale]
fs_BOLD_4 = fs_BOLD_4[scanning_timescale]
fs_BOLD_d = fs_BOLD_d[scanning_timescale]
fr_BOLD_s = fr_BOLD_s[scanning_timescale]
fr_BOLD_4 = fr_BOLD_4[scanning_timescale]
fr_BOLD_d = fr_BOLD_d[scanning_timescale]
# remove first few scans from BOLD signal array and from BOLD timescale array
mr_time = np.delete(mr_time, np.arange(scans_to_remove))
v1_BOLD_s = np.delete(v1_BOLD_s, np.arange(scans_to_remove))
v1_BOLD_4 = np.delete(v1_BOLD_4, np.arange(scans_to_remove))
v1_BOLD_d = np.delete(v1_BOLD_d, np.arange(scans_to_remove))
v4_BOLD_s = np.delete(v4_BOLD_s, np.arange(scans_to_remove))
v4_BOLD_4 = np.delete(v4_BOLD_4, np.arange(scans_to_remove))
v4_BOLD_d = np.delete(v4_BOLD_d, np.arange(scans_to_remove))
it_BOLD_s = np.delete(it_BOLD_s, np.arange(scans_to_remove))
it_BOLD_4 = np.delete(it_BOLD_4, np.arange(scans_to_remove))
it_BOLD_d = np.delete(it_BOLD_d, np.arange(scans_to_remove))
d1_BOLD_s = np.delete(d1_BOLD_s, np.arange(scans_to_remove))
d1_BOLD_4 = np.delete(d1_BOLD_4, np.arange(scans_to_remove))
d1_BOLD_d = np.delete(d1_BOLD_d, np.arange(scans_to_remove))
d2_BOLD_s = np.delete(d2_BOLD_s, np.arange(scans_to_remove))
d2_BOLD_4 = np.delete(d2_BOLD_4, np.arange(scans_to_remove))
d2_BOLD_d = np.delete(d2_BOLD_d, np.arange(scans_to_remove))
fs_BOLD_s = np.delete(fs_BOLD_s, np.arange(scans_to_remove))
fs_BOLD_4 = np.delete(fs_BOLD_4, np.arange(scans_to_remove))
fs_BOLD_d = np.delete(fs_BOLD_d, np.arange(scans_to_remove))
fr_BOLD_s = np.delete(fr_BOLD_s, np.arange(scans_to_remove))
fr_BOLD_4 = np.delete(fr_BOLD_4, np.arange(scans_to_remove))
fr_BOLD_d = np.delete(fr_BOLD_d, np.arange(scans_to_remove))

In [19]:
# Save the downsampled BOLD data for display purposes
# round of mr time for display purposes
mr_time = np.round(mr_time, decimals=0)

# create a numpy array of timeseries
lsnm_BOLD = np.array([mr_time,v1_BOLD_s, v1_BOLD_4, v1_BOLD_d,
		      v4_BOLD_s, v4_BOLD_4, v4_BOLD_d,
                      it_BOLD_s, it_BOLD_4, it_BOLD_d,
                      fs_BOLD_s, fs_BOLD_4, fs_BOLD_d,
                      d1_BOLD_s, d1_BOLD_4, d1_BOLD_d, 
                      d2_BOLD_s, d2_BOLD_4, d2_BOLD_d,
                      fr_BOLD_s, fr_BOLD_4, fr_BOLD_d
                      ])
BOLD_file = 'WK_laminar_LSNM_bold_balloon_7T_drain_DMS.npy'
np.save(BOLD_file, lsnm_BOLD)